In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [3]:
df = pd.read_excel(r'/Users/dominikculo/downloads/velocity_of_molecule.xlsx', usecols='A,B,C,D,E')
m1 = pd.DataFrame (df, columns=['m1']).dropna()
m2 = pd.DataFrame (df, columns=['m2']).dropna()
m3 = pd.DataFrame (df, columns=['m3']).dropna()
m4 = pd.DataFrame (df, columns=['m4']).dropna()
m_open = pd.DataFrame (df, columns=['m_open']).dropna()

In [5]:
m1 = np.array(m1)
m2 = np.array(m2)
m3 = np.array(m3)
m4 = np.array(m4)
m_open = np.array(m_open)

m1_err = math.sqrt((np.std(m1)/len(m1))**2 + 0.001**2)
m2_err = math.sqrt((np.std(m2)/len(m2))**2 + 0.001**2)
m3_err = math.sqrt((np.std(m3)/len(m3))**2 + 0.001**2)
m4_err = math.sqrt((np.std(m4)/len(m4))**2 + 0.001**2)
m_open_err = math.sqrt((np.std(m_open)/len(m_open))**2 + 0.001**2)

m_lid = [m1[i] - m2[i] for i in range(len(m1))]
m_lid_err = math.sqrt(m1_err**2 + m2_err**2)

m_LL = [m3[i] - m4[i] for i in range(len(m3))]
m_LL_err = math.sqrt(m3_err**2 + m4_err**2)

time = np.linspace(0, 300, 11)

#this isn't proper, but it gives the same value as the proper way, so I'll leave it
delta_m_list = [m_lid[i] - m_LL[i] for i in range(len(m_lid))]
delta_m = sum(delta_m_list)/len(delta_m_list)
print(delta_m)
delta_m_err = math.sqrt(m_LL_err**2 + m_lid_err**2)
print(delta_m_err)

[0.0056]
0.007076270203998748


In [ ]:
def least_squares(x, y, sig_y, sig_x=0.25):
    N = len(x)
    sum_x = sum(x)
    sum_x2 = sum([x[i]**2 for i in range(len(x))])
    sum_y = sum(y)
    sum_xy = sum([x[i] * y[i] for i in range(len(x))])
    
    delta = N * sum_x2 - sum_x**2
    A = (sum_x2 * sum_y - sum_x * sum_xy)/delta
    B = (N * sum_xy - sum_x * sum_y)/delta
    
    ysig = math.sqrt(abs((1/(1 - N)) * sum([(y[i] - A - B * x[i])**2 for i in range(len(x))])))
    A_sig = ysig * math.sqrt(sum_x2/delta)
    B_sig = ysig * math.sqrt(N/delta)
    
    print(ysig)
    
    return A, B, A_sig, B_sig

A, B, A_err, B_err = least_squares(time, m_open, m_open_err)
print(A, B, A_err, B_err)

In [ ]:
mean_vel = (2 * delta_m * 9.81)/abs(B)
mean_vel_err = 2* 9.81 * math.sqrt((B_err/B)**2 + (delta_m_err/delta_m)**2)

print(float(mean_vel), '+/-', mean_vel_err)

In [ ]:
y_err = math.sqrt(m_open_err**2 + (B**2 * 0.25**2))

plt.plot(time, [A + B * time[i] for i in range(len(time))])
for i in range(len(time)):
    plt.errorbar(time[i], m_open[i], marker='o', color='black', yerr=y_err)
    
plt.title('Change in mass over time')
plt.xlabel('time(s)')
plt.ylabel('mass(g)')

plt.savefig('mean_velocity_plot', bbox_inches='tight')